# 1- Implementar funções em Python que façam a codificação/decodificação do código Aritmético e do Código ANS, descrito no material anexo, e verificando com casos de teste (incluindo os exemplos do texto) se as funções estão operando corretamente. Mandatório explicar a lógica de sua implementação e dos testes.

In [24]:
import numpy as np
import math

# Definir uma função auxiliar para calcular a entropia de uma distribuição de probabilidade
def entropy(pmf):
    return -sum(p * math.log2(p) for p in pmf if p > 0)

# Definir uma função auxiliar para calcular a função de distribuição cumulativa (cdf) a partir da função de massa de probabilidade (pmf)
def cdf(pmf):
    return np.cumsum(pmf)

# Definir uma função auxiliar para encontrar o símbolo correspondente a um valor de cdf, usando busca binária
def find_symbol(cdf, value):
    return np.searchsorted(cdf, value, side="right") - 1

# Definir a função principal para codificar uma mensagem usando o código aritmético
def arithmetic_encode(message, pmf, precision):
    # Calcular a cdf a partir da pmf
    cdf_values = cdf(pmf)
    # Inicializar o intervalo de codificação como [0, 1)
    low, high = 0, 1
    # Iterar sobre os símbolos da mensagem
    for symbol in message:
        # Obter a cdf baixa e alta do símbolo
        cdf_low, cdf_high = cdf_values[symbol], cdf_values[symbol + 1]
        # Atualizar o intervalo de codificação de acordo com a cdf do símbolo
        low = low + (high - low) * cdf_low
        high = low + (high - low) * (cdf_high - cdf_low)
    # Retornar o ponto médio do intervalo final como o número de codificação
    return (low + high) / 2

# Definir a função principal para decodificar uma mensagem usando o código aritmético
def arithmetic_decode(encoded_number, pmf, message_length):
    # Calcular a cdf a partir da pmf
    cdf_values = cdf(pmf)
    # Inicializar o intervalo de decodificação como [0, 1)
    low, high = 0, 1
    # Inicializar a mensagem decodificada como uma matriz de zeros
    decoded_message = np.zeros(message_length, dtype=np.int32)
    # Iterar sobre os símbolos da mensagem
    for i in range(message_length):
        # Encontrar o símbolo correspondente ao valor de codificação usando busca binária
        symbol = find_symbol(cdf_values, encoded_number)
        # Adicionar o símbolo à mensagem decodificada
        decoded_message[i] = symbol
        # Obter a cdf baixa e alta do símbolo
        cdf_low, cdf_high = cdf_values[symbol], cdf_values[symbol + 1]
        # Atualizar o intervalo de decodificação de acordo com a cdf do símbolo
        low = low + (high - low) * cdf_low
        high = low + (high - low) * (cdf_high - cdf_low)
    # Retornar a mensagem decodificada
    return decoded_message

# Função auxiliar para converter uma mensagem codificada para uma lista de palavras comprimidas
def convert_to_compressed_list(encoded_message, precision):
    compressed_list = []
    for _ in range(precision):
        encoded_message *= 2
        bit = int(encoded_message)
        compressed_list.append(bit)
        encoded_message -= bit
    return compressed_list

# Função auxiliar para converter uma lista de palavras comprimidas de volta para o número codificado
def convert_to_encoded_number(compressed_list):
    encoded_number = 0
    for bit in compressed_list:
        encoded_number = encoded_number * 2 + bit
    return encoded_number

# Definir a função principal para codificar uma mensagem usando o código ANS
def ans_encode(message, pmf, precision):
    # Calcular a cdf a partir da pmf
    cdf_values = cdf(pmf)
    # Inicializar o estado interno como 0
    state = 0
    # Inicializar a lista de palavras comprimidas
    compressed_list = []
    # Iterar sobre os símbolos da mensagem
    for symbol in message:
        # Obter a cdf baixa e alta do símbolo
        cdf_low, cdf_high = cdf_values[symbol], cdf_values[symbol + 1]
        # Calcular a nova posição do estado interno
        state = (state >> precision) * (cdf_high - cdf_low) + (state % (1 << precision)) + int(cdf_low * (1 << precision))
        # Adicionar os bits mais significativos do estado à lista de palavras comprimidas
        compressed_list.extend(convert_to_compressed_list(state >> (precision - 8), 8))
        # Atualizar o estado interno
        state = state & ((1 << precision) - 1)
    # Retornar o estado final e a lista de palavras comprimidas
    return state, compressed_list

# Definir a função principal para decodificar uma mensagem usando o código ANS
def ans_decode(state, compressed_list, pmf, message_length):
    # Calcular a cdf a partir da pmf
    cdf_values = cdf(pmf)
    # Inicializar a mensagem decodificada como uma matriz de zeros
    decoded_message = np.zeros(message_length, dtype=np.int32)
    # Iterar sobre os símbolos da mensagem
    for i in range(message_length):
        # Converter os próximos 8 bits da lista de palavras comprimidas de volta para o número codificado
        encoded_number = convert_to_encoded_number(compressed_list[:8])
        compressed_list = compressed_list[8:]
        # Calcular a nova posição do estado interno
        state = (state >> 8) * (cdf_values[-1] - cdf_values[0]) + (state % 256) + int(cdf_values[0] * 256) + encoded_number
        # Encontrar o símbolo correspondente ao valor de codificação usando busca binária
        symbol = find_symbol(cdf_values, state)
        # Adicionar o símbolo à mensagem decodificada
        decoded_message[i] = symbol
        # Obter a cdf baixa e alta do símbolo
        cdf_low, cdf_high = cdf_values[symbol], cdf_values[symbol + 1]
        # Atualizar o estado interno
        state = (state - int(cdf_low * 256)) & 0xFF
    # Retornar a mensagem decodificada
    return decoded_message

# Caso de teste 1: mensagem "BARCA VELHA" com pmf [0.1, 0.1, 0.1, 0.1, 0.1, 0]
message = np.array([1, 0, 2, 3, 0, 4, 5, 2, 3, 0], dtype=np.int32)
pmf = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0], dtype=np.float32)
precision = 16
encoded_message = ans_encode(message, pmf, precision)
decoded_message = ans_decode(encoded_message, pmf, len(message))
print("Caso de teste 1:")   
print("Mensagem original:", message)
print("Mensagem codificada:", encoded_message)
print("Mensagem decodificada:", decoded_message)
print("Entropia da mensagem original:", entropy(pmf))
print("Comprimento da mensagem original:", len(message))
print("Comprimento da mensagem codificada:", len(convert_to_compressed_list(encoded_message, precision)))
print("Comprimento da mensagem decodificada:", len(decoded_message))
print("")



TypeError: ufunc 'right_shift' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

# 2-Comparar a eficiência de compressão entres os três códigos estatísticos estudados: Huffman, Aritmético e ANS.

In [14]:
import numpy as np
import math
import heapq
from collections import Counter
import numpy as np

# Huffman coding
def huffman_encode(message):
    # Count the frequency of each symbol in the message
    freq = Counter(message)
    # Build the Huffman tree
    heap = [[weight, [symbol, ""]] for symbol, weight in freq.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    huffman_tree = sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[-1]), p))
    # Encode the message using the Huffman tree
    encoded_message = ''.join([symbol[1] for symbol in huffman_tree])
    return encoded_message

def huffman_decode(encoded_message, huffman_tree):
    # Decode the message using the Huffman tree
    decoded_message = ""
    current_code = ""
    for bit in encoded_message:
        current_code += bit
        for symbol in huffman_tree:
            if current_code == symbol[1]:
                decoded_message += symbol[0]
                current_code = ""
                break
    return decoded_message

# Arithmetic coding
def arithmetic_encode(message, pmf, precision):
    # Calculate the cumulative distribution function (cdf) from the probability mass function (pmf)
    cdf_values = np.cumsum(pmf)
    # Initialize the encoding interval as [0, 1)
    low, high = 0, 1
    # Iterate over the symbols in the message
    for symbol in message:
        # Get the low and high cdf values of the symbol
        cdf_low, cdf_high = cdf_values[symbol], cdf_values[symbol + 1]
        # Update the encoding interval according to the symbol's cdf
        low = low + (high - low) * cdf_low
        high = low + (high - low) * (cdf_high - cdf_low)
    # Return the midpoint of the final interval as the encoded number
    return (low + high) / 2

def arithmetic_decode(encoded_number, pmf, message_length):
    # Calculate the cumulative distribution function (cdf) from the probability mass function (pmf)
    cdf_values = np.cumsum(pmf)
    # Initialize the decoding interval as [0, 1)
    low, high = 0, 1
    # Initialize the decoded message as an array of zeros
    decoded_message = np.zeros(message_length, dtype=np.int32)
    # Iterate over the symbols in the message
    for i in range(message_length):
        # Find the symbol corresponding to the encoding value using binary search
        symbol = np.searchsorted(cdf_values, encoded_number, side="right") - 1
        # Add the symbol to the decoded message
        decoded_message[i] = symbol
        # Get the low and high cdf values of the symbol
        cdf_low, cdf_high = cdf_values[symbol], cdf_values[symbol + 1]
        # Update the decoding interval according to the symbol's cdf
        low = low + (high - low) * cdf_low
        high = low + (high - low) * (cdf_high - cdf_low)
    # Return the decoded message
    return decoded_message

# ANS coding
def ans_encode(message, pmf, precision):
    # Calculate the cumulative distribution function (cdf) from the probability mass function (pmf)
    cdf_values = np.cumsum(pmf)
    # Initialize the internal state as 0
    state = 0
    # Initialize the compressed list
    compressed_list = []
    # Iterate over the symbols in the message
    for symbol in message:
        # Get the low and high cdf values of the symbol
        cdf_low, cdf_high = cdf_values[symbol], cdf_values[symbol + 1]
        # Calculate the new position of the internal state
        state = (state >> precision) * (cdf_high - cdf_low) + (state % (1 << precision)) + int(cdf_low * (1 << precision))
        # Add the most significant bits of the state to the compressed list
        compressed_list.extend([int(bit) for bit in format(state >> (precision - 8), '08b')])
        # Update the internal state
        def arithmetic_encode(message, pmf, precision):
            # Arithmetic encoding logic here
            # ...
            return encoded_message

        # Test case
        message = "ABRACADABRA"
        pmf = np.array([0.3, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05], dtype=np.float32)
        precision = 16

        # Huffman coding
        huffman_encoded_message = huffman_encode(message)
        huffman_compression_ratio = len(huffman_encoded_message) / (len(message) * 8)

        # Arithmetic coding
        arithmetic_encoded_message = arithmetic_encode([ord(symbol) for symbol in message], pmf, precision)
        arithmetic_compression_ratio = len(arithmetic_encoded_message) / (len(message) * 8 * precision)

        # ANS coding
        ans_encoded_state, ans_encoded_message = ans_encode([ord(symbol) for symbol in message], pmf, precision)
        ans_compression_ratio = len(ans_encoded_message) / (len(message) * 8 * precision)

        print("Compression ratios:")
        print("Huffman coding:", huffman_compression_ratio)
        print("Arithmetic coding:", arithmetic_compression_ratio)
        print("ANS coding:", ans_compression_ratio)
